In [5]:
from lnas import LnasFormat
import pathlib
import numpy as np

NORMAL_OFFSET = 0.125

block_lbl = "G300"

meshes_path = pathlib.Path(f"/mnt/dev01_openfoam_data/insight-volume/PrologisCajamar4_new_version/{block_lbl}/mesh")
mesh_path = meshes_path / f"{block_lbl}.lnas"
neg_mesh_path = mesh_path.parent / f"{block_lbl}_negative.lnas"
output_path = mesh_path.parent / f"{block_lbl}.merged.lnas"

mesh = LnasFormat.from_file(mesh_path)
neg_mesh = LnasFormat.from_file(neg_mesh_path)

geometry_negative = neg_mesh.geometry
geometry_negative.triangles = np.flip(geometry_negative.triangles, 1)
geometry_negative._full_update()
geometry_negative.vertices = geometry_negative.vertices + geometry_negative.vertices_normals * NORMAL_OFFSET

mesh.join(lnas_fmts=[neg_mesh], surfaces_suffixes=["_inside"])
mesh.to_file(output_path)

In [7]:
import pandas as pd
import pathlib

case_dict = {
    "G100" : ["013", "058", "103", "148", "193", "238", "283", "328"],
    "G200" : ["000", "045", "090", "135", "180", "225", "270", "315"],
    "G300" : ["000", "045", "090", "135", "180", "225", "270", "315"],
}

for block_lbl, case_list in case_dict.items():
    for case_lbl in case_list:
        data_path = pathlib.Path(f"/mnt/dev01_openfoam_data/insight-volume/PrologisCajamar4_new_version/{block_lbl}/cases/{case_lbl}/data/pressure_measure")

        body_hist_series = pd.read_hdf(data_path / f"bodies.{block_lbl}_hs.data.h5")
        negative_body_hist_series = pd.read_hdf(data_path / f"bodies.{block_lbl}_neg_hs.data.h5")

        last_index = body_hist_series.point_idx.max()
        negative_body_hist_series.point_idx += last_index + 1

        new_df = pd.concat([body_hist_series, negative_body_hist_series]).sort_values(
            by=["time_step", "point_idx"]
        )
        new_df.to_hdf(path_or_buf=data_path.parent / f"bodies.{block_lbl}.merged.data.h5", key="df", mode="w")